In [1]:
import pandas as pd
import bs4
import os

In [2]:
data = pd.read_csv('./laurier.csv')

In [3]:
# data['content'] = data['raw'].apply(lambda x: str(bs4.BeautifulSoup(x, 'html.parser').select_one('div.content')))

In [3]:
from urllib.parse import urlparse, parse_qs

def getCourseData(url: str, content: str):
    soup = bs4.BeautifulSoup(content, 'html.parser')
    
    header = soup.select_one('div.content > h1').contents

    if len(header) < 2:
        return {
            'url' : url,
            'title': None,
            'name': None,
            'credits': None,
            'description': None
        }

    title = header[0].text.strip()
    name = header[2].text.strip()
    credits = header[4].text.strip()
    description = soup.select_one('div.content > p')
    params = parse_qs(urlparse(url).query)
    

    return {
        'url' : url,
        'title': title,
        'name': name,
        'credits': credits,
        'description': description.text.strip() if description else None,
        **{key : value[0] for key, value in params.items()}
    }

In [4]:
content = []
for row in data[data['url'].str.startswith('https://academic-calendar.wlu.ca/course.php')].itertuples():
    content.append(getCourseData(row.url, row.content))

courses = pd.DataFrame(content).dropna()

In [7]:
departments = {
    1037: 'Faculty of Music',
    1046: 'Martin Luther University College',
    1033: 'Faculty of Arts',
    1035: 'Faculty of Liberal Arts',
    1047: 'Faculty of Human and Social Sciences',
    1036: 'Lazaridis School of Business and Economics',
    1034: 'Faculty of Science',
    1048: 'Lyle S Hallman Faculty of Social Work',
    1043: 'Faculty of Education',
    1032: 'Interdisciplinary Course Offerings'
}

In [38]:
courses.rename(columns={'title' : 'code'}).to_csv('./courses.csv', index=False)

In [39]:
courses.columns

Index(['url', 'title', 'name', 'credits', 'description', 'c', 'cal', 'd', 's',
       'y', 'department'],
      dtype='object')